# Importing Packages and API Setup

In [ ]:
# Only needs to be run upon first-time use
import sys
!{sys.executable} -m pip install alpha_vantage
!{sys.executable} -m pip install quandl

In [ ]:
import numpy as np
import scipy.stats as stats
import itertools as it
import pandas as pd
import cmath as c
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import datetime as dt
import quandl as q
import time
# https://github.com/RomelTorres/alpha_vantage
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators 
from alpha_vantage.sectorperformance import SectorPerformances

In [ ]:
#Alpha Vantage API keys
API_URL = 'https://www.alphavantage.co/query?'
API_KEY1 = 'TMANNBF5NO707ZEI'
API_KEY2 = 'AO48IFCXLA3BX1O9'
API_KEY3 = 'GDJZQ31AX2TZRFA1'
API_KEY4 = 'APHZP2X3GZ7UQ7NO'
API_KEY5 = 'X9LEOT46QKZHC0O9'

# Quandl API key
q.ApiConfig.api_key = "ntUKZwApL5qnmU9pk_rB"

# Output formats: json dictionaries (default), DataFrame, csv.
# csv does not support ForeignExchange, SectorPerformances, TechIndicators.
# indexing_type can also be integer (default: date)
ts_dct = [TimeSeries(key=API_KEY1, output_format='pandas', indexing_type='date'), 
       TimeSeries(key=API_KEY2, output_format='pandas', indexing_type='date'), 
       TimeSeries(key=API_KEY3, output_format='pandas', indexing_type='date'),
       TimeSeries(key=API_KEY4, output_format='pandas', indexing_type='date'),
       TimeSeries(key=API_KEY5, output_format='pandas', indexing_type='date')]

ti_dct = [TechIndicators(key=API_KEY1, output_format='pandas', indexing_type='date'), 
       TechIndicators(key=API_KEY2, output_format='pandas', indexing_type='date'), 
       TechIndicators(key=API_KEY3, output_format='pandas', indexing_type='date'),
       TechIndicators(key=API_KEY4, output_format='pandas', indexing_type='date'),
       TechIndicators(key=API_KEY5, output_format='pandas', indexing_type='date')]

sp_dct = [SectorPerformances(key=API_KEY1, output_format='pandas', indexing_type='date'), 
       SectorPerformances(key=API_KEY2, output_format='pandas', indexing_type='date'), 
       SectorPerformances(key=API_KEY3, output_format='pandas', indexing_type='date'),
       SectorPerformances(key=API_KEY4, output_format='pandas', indexing_type='date'),
       SectorPerformances(key=API_KEY5, output_format='pandas', indexing_type='date')]

# Data Collection Methods

This is the primary method for retrieving data from the AlphaVantage API. Rather than worrying about all of the individual data collection methods, this method allows the user to just specify the key parameters, and the appropriate API method retrieves the data. Because the API only allows for 5 calls/minute, each block of ticker calls can be made one at a time every minute. Thus, **`collect()` is HIGHLY RECOMMENDED** as it prevents errors related to exceeding the 5 calls/minute rate.

In [ ]:
global port
global port_rets

port = {} # Stores all stock data
port_rets = {} # Stores all return data

# Used to get individual stock data from the AlphaVantage API.
# symbols: list of stock tickers as strings
# start_date, end_date: string of date 'YYYY-MM-DD'
# intra_int: intraday interval; '1min', '5min', '15min', '30min', '60min' (default '15min')
# add: by default adds the searched asset to the portfolio.
# api_num: index to reference of ts_dct; relevant when pulling more than 5 stocks at once
def get_data(symbol, start_date= None, end_date= None, 
             int_type='daily_adjusted', intra_int='15min', add=True, api_key=ts_dct[0]):
    
    if int_type is 'daily_adjusted':
        data, meta = api_key.get_daily_adjusted(symbol=symbol, outputsize='full')
    else: 
        data, meta = api_key.get_intraday(symbol=symbol, interval=intra_int, outputsize='full')

    # Remove numbers from column labels.
    data.rename(columns=lambda col: col[3:], inplace=True)

    # Filter the DataFrame to contain only those specified by the start and end dates.
    if start_date and end_date:
        data = data.loc[start_date:end_date]
    elif start_date:
        data = data.loc[start_date:]
    else:
        data = data.loc[:end_date]

    # Add the stock data to a DataFrame and reindex to be DateTime indices.
    df_stock = pd.DataFrame(data)
    df_stock.index = pd.to_datetime(df_stock.index)

    # By default, add to the portfolio
    if add:
        port[symbol] = df_stock

    return pd.DataFrame(df_stock)

In [ ]:
# Used to calculate the daily percent change, log returns, and
# cumulative returns ovhttp://localhost:8888/notebooks/Desktop/Hesiod%20Financial/LKQ%20(April%2014%202019).ipynb#er time for each stock in the portfolio.
# portfolio: may refer to portfolio containing all stocks or an 
# individual stock
# symbol: specify ticker as string if calculating an individual 
# stock's returns not already in the portfolio
def calculate_returns(portfolio, symbol=None):

    # True if the portfolio is a dictionary containing multiple stocks.
    if type(portfolio) is type({1:2}):
        for symb, data in portfolio.items():

            # Get price close data.
            daily_close = data['close']

            # Calculate percent change for each period.
            daily_pct_change = daily_close.pct_change()
            daily_pct_change.fillna(0, inplace=True)

            # Calculate log returns for each period.
            daily_log_returns = np.log(daily_close.pct_change()+1)
            daily_log_returns.fillna(0, inplace=True)

            # Cumulative daily rate of return is useful to determine the 
            # value of an investment at regular intervals.
            cum_daily_return = (1 + daily_pct_change).cumprod()
            port_rets[symb] = pd.DataFrame({'pct change': daily_pct_change, 
                                       'log returns':daily_log_returns, 
                                       'cumulative daily':cum_daily_return})
        return port_rets[symb]

    # True if only calculating for one stock.
    else:
        daily_close = portfolio['close']
        daily_pct_change = daily_close.pct_change()
        daily_pct_change.fillna(0, inplace=True)

        daily_log_returns = np.log(daily_close.pct_change()+1)
        daily_log_returns.fillna(0, inplace=True)

        # Cumulative daily rate of return is useful to determine 
        # the value of an investment at regular intervals.
        cum_daily_return = (1 + daily_pct_change).cumprod()
        port_rets[symbol] = pd.DataFrame({'pct change': daily_pct_change, 
                                       'log returns':daily_log_returns, 
                                       'cumulative daily':cum_daily_return})
        return port_rets[symbol]

In [ ]:
# Primary method for acquring stock data. Recommended over get_data() because
# it avoids the 5 API calls/min limitation by inherently waiting 1 minute if
# more than 5 stocks are being called at once.
# tickers: list of ticker strings
# start_date, end_date: string of date 'YYYY-MM-DD'
def collect(tickers, start=None, end=None, add_to_port=True):   
    # Partition list of tickers into sub-lists of 4 tickers.
    grouped_ticks = [tickers[i*5:(i+1)*5] for i in range(
        (len(tickers)+ 4)// 5)]
    # If there are more than 1 paritions, call each partition
    # one at a time and use one API key to get the data.
    if 1 < len(grouped_ticks) <= len(ts_dct):
        for group in grouped_ticks:
            sub_collect(group=group, add_to_port=add_to_port)
    elif len(grouped_ticks) > len(ts_dct):
        print(f'Max number of tickers callable at once: {len(ts_dct)*5}')
    else:
        sub_collect(group=grouped_ticks.pop(), add_to_port=add_to_port)
# Helper method for collect(). Iterates through list of grouped tickers
# and calls to get the stock data for each ticker.
def sub_collect(group, add_to_port=True):
    for g, api in zip(group, ts_dct):
        get_data(symbol=g, start_date=start, end_date=end, add=add_to_port, api_key=api)
        time.sleep(6)

# Construct the Portfolio and Timeframe

In [ ]:
tickers = ['EPD', 'HRTX', 'PAGS', 'QQQ', 'COMM', 'ACM', 'SPY', 'LKQ', 'ZUO']

today = dt.datetime.today()

# Enter start date (default set to 1 year ago)
start = dt.datetime(today.year - 1, today.month, today.day).strftime('%Y-%m-%d')

# Default set to today
end = today.strftime('%Y-%m-%d')

collect(tickers, start, end)

In [ ]:
port.keys()

In [ ]:
calculate_returns(port);

# Plotting

In [ ]:
# ticker: stock symbol
# price_type: 'adjusted close' (default), 'close', 'open'
# Returns matplotlib objects as well
def plot_returns(ticker, price_type='adjusted close'):

    # Plot price action
    price = port[ticker][price_type].plot(
        figsize=(15,3), label=ticker, legend=False)
    plt.title(f'{ticker} Historical Price Action')
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')
    plt.show()

    # Plot daily pct change
    pct_change = port_rets[ticker]['pct change'].plot(
        figsize=(15,3), label=ticker,legend=False)
    plt.title(f'{ticker} Daily Percent Change')
    plt.xlabel('Date')
    plt.ylabel('Daily Percent Return')
    plt.show()

    # Plot cumulative returns
    cum_daily = port_rets[ticker]['cumulative daily'].plot(
        figsize=(15,3), label=ticker,legend=False)
    plt.title(f'{ticker} Cumulative Daily Returns')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Percent Return')
    plt.show()

    return price, pct_change, cum_daily

In [ ]:
plot_returns('ZUO');

# Statistical Measurements

In [ ]:
def plot_cdf(ticker):
    
    r = port_rets[ticker]['pct change']  
    ax = sns.distplot(r,
                      rug=True,
                      hist_kws=dict(cumulative=True), 
                      kde_kws=dict(cumulative=True))
    ax.set_title(ticker + ' CDF')
    ax.set_ylabel('Density')
    ax.set_xlabel('% Change of Returns')
    plt.show()
    
    moments = pd.DataFrame(columns=['Mean', 
                                    'SD', 
                                    'Skew', 
                                    'Kurtosis',
                                   'Normal (JB)'])
    
    vals = pd.Series([float('%.3f'%(r.mean())),
                         float('%.3f'%(r.std())),
                         float('%.3f'%(r.skew())),
                         float('%.3f'%(r.kurtosis())),
                         stats.jarque_bera(r)[1] > 0.05],
                        index=moments.columns).rename('')
    moments = moments.append(vals)
    
    return moments

In [ ]:
plot_cdf('ZUO')

In [ ]:
# Plot the volatility (i.e. standard deviation) of an asset in the portfolio.
# show_price: if True, also plots price data of the stock.
# show_benchmark: if True, also plots volatility of S&P 500.
def plot_volatility(ticker, min_periods=25, show_price=False, show_bench=False):
    
    global price, bench_vol
    price= None
    bench_vol= None

    # Get the asset volatility and optionally the S&P volatility and price data of the asset.    
    vol = port_rets[ticker]['pct change'].rolling(min_periods).std()*np.sqrt(min_periods)
    if show_bench:
        start = str(port_rets[ticker].index[0])[:-9]
        end = str(port_rets[ticker].index[-1])[:-9]
        bench_data = port['SPY']
        bench_rets = port_rets['SPY']
        bench_vol = bench_rets['pct change'].rolling(min_periods).std()*np.sqrt(min_periods)
    if show_price:
        price = port[ticker]['close']
    
    fig, ax1 = plt.subplots(figsize=(12,3))
    
    vol_color = 'tab:red'
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Percent')
    ax1.plot(vol, color=vol_color, label=ticker)
    ax1.legend(loc=1)
    
    if show_bench:
        bench_color='tab:orange'
        ax1.plot(bench_vol, color=bench_color, label='S&P')
        ax1.legend(loc=1)
    if show_price:
        ax2 = ax1.twinx() # second axes with same x-axis
        price_color = 'tab:blue'
        ax2.set_ylabel(f'{ticker} Price')
        ax2.plot(price, color=price_color, label=f'{ticker} price')
        ax2.tick_params(axis='y', labelcolor=price_color)
        ax2.legend(loc=2)
    vol.dropna()
    asset_mean = vol.mean()
    asset_std_dev = vol.std()
    asset_std_err = vol.sem()
    
    bench_mean = bench_vol.mean()
    bench_std_dev = bench_vol.std()
    bench_std_err = bench_vol.sem()
    
    fig.tight_layout()
    plt.title(f'{ticker} vs. S&P: Volatility ({min_periods}-Day Rolling Period)')
    plt.show()
        
    vol_stats = pd.DataFrame(index=['mean', 'std dev', 'std err'])
    vol_stats[ticker] = pd.Series((asset_mean, asset_std_dev, asset_std_err), index=vol_stats.index)
    vol_stats['S&P 500'] = pd.Series((bench_mean, bench_std_dev, bench_std_err), index=vol_stats.index)
    
    agree=False
    if asset_mean <= bench_mean and asset_mean + asset_std_err >= bench_mean:
        agree = True
    elif asset_mean >= bench_mean and asset_mean - asset_std_err <= bench_mean:
        agree=True
    
    print(vol_stats) 
    print(f'S&P and {ticker} agree within std error: {agree}')

In [ ]:
plot_volatility('EPD', min_periods=20, show_bench=True)

In [ ]:
start = str(list(port_rets.values())[0].index[0])[:-9]
end = str(list(port_rets.values())[0].index[-1])[:-9]

In [ ]:
def plot_linreg(ticker_1, ticker_2, 
                price_type='close',
                visible=True):
    
    params = stats.linregress(port[ticker_1][price_type],
                    port[ticker_2][price_type])
        
    if visible:
        ax = sns.regplot(x=port[ticker_1][price_type], 
                y=port[ticker_2][price_type])
        ax.set_xlabel(ticker_1)
        ax.set_ylabel(ticker_2)
        start = port[ticker_1].index[0].strftime('%Y-%m-%d')
        end = port[ticker_1].index[-1].strftime('%Y-%m-%d')
        ax.set_title(f'{ticker_1}-{ticker_2} LR ({start} - {end})')
        plt.show()
    
    vals = pd.DataFrame(columns=['R squared', 'p value', 'std err'])
    
    v = pd.Series([float('%.3f'%(params.rvalue**2)), 
                   float('%.3f'%(params.pvalue)), 
                   float('%.3f'%(params.stderr))],
                  index=vals.columns).rename('')
    vals = vals.append(v)
    return vals

In [ ]:
def plot_corr_heatmap():
    
    col = []
    
    for sym, data in port.items():
        row = []
        
        for s, d in port.items():
            r = np.corrcoef(d['close'], data['close'])[0][1]
            row.append(r)
    
        col.append(row)
        
    corr_matrix = pd.DataFrame(col, columns=port.keys())
    corr_matrix.index = port.keys()
    
    mask = np.zeros_like(corr_matrix)
    mask[np.triu_indices_from(mask)]=True
    ax = sns.heatmap(corr_matrix, 
                     annot=True,
                     mask=mask,
                     cmap='RdYlGn_r')
    ax.set_title('Portfolio Correlation Heat Map')
    return corr_matrix

In [ ]:
plot_corr_heatmap()

In [ ]:
def plot_scatter_matrix():
    
    close = pd.DataFrame()
    for s, d in port.items():
        close[s] = port[s]['close']
    #pd.plotting.scatter_matrix(close, 
    #                           alpha=0.2,
    #                           figsize=(10,10), 
    #                           diagonal='kde');
        # plt.title('Portfolio Scatter Matrix')
        # plt.show()
    ax = sns.pairplot(close)
    plt.show()

In [ ]:
plot_scatter_matrix()

## Adjusted for Skewness and Kurtosis Sharpe Ratio (ASKSR)

This is a gross oversimplification of the problem that arises with using the standard mean-variance-based Sharpe Ratio. When using the Sharpe Ratio to determine the "risk-reward" value of owning an asset or portfolio of assets, it is assumed that the return data is normally distributed. However, this is almost never the case, so the value of the Sharpe Ratio can be misleading. To account for this, Zakamouline and Koekebakker (2008) authored an article deriving a parametric form of a special-case form of the generalized Sharpe Ratio that accounts for higher-order moments of distribution than just mean and variance. That is, the returns are not assumed to be normally disributed, so the ASKSR accounts for skewness (3rd moment) and kurtossis (4th moment) of the returns distribution of an asset. To read more into the "nitty-gritty," click on the link below.

https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1028715

The authors chose to use the normal inverse Gaussian distribution (NIGD) because:
1. NIGD has an explicit expression for the probability density function
2. Distributions of risky asset returns can be often fitted extremely well by the NIGD
3. For the NIGD, there are explicit formulas for finding the parameters of the distribution via the values of the first four moments of the distribution.

$\eta$ and $\delta$ are ordinary paramters of location and scale whereas $\alpha$ and $\beta$ determine the shape of the density. In particular, $\beta$ determines the degree of skewwness.

For symmetrical densities, $\beta = 0$. The conditions for a viable NIG density are $\delta > 0$, $\alpha > 0$, and $\frac{\vert \beta \vert}{\alpha} < 1$. Note that to get meaningful parameters of the NIGD, the following conditions must be satisfied:

\begin{align}
K > 3 + \frac{5}{3}S^2
\end{align}

* Mean: $\mu = \eta + \delta\frac{\beta}{\varphi}$

* Variance: $\sigma^2 = \delta\frac{\alpha^2}{\varphi^3}$

* Skew: $S = 3\frac{\beta}{\alpha \sqrt{\delta\varphi}}$

* Kurtosis: $K = 3 + \frac{3}{\delta\varphi}\left( 1 + 4\left(\frac{\beta}{\alpha} \right)^2 \right)$

\begin{align}
ASKSR &= \sqrt{2\left(\lambda a^*(\eta - r) -\delta\left (\varphi - \sqrt{\alpha^2 - (\beta - \lambda a^*)^2}\right)\right)} \\\\
\end{align}

\begin{align}
a^* &= \frac{1}{\lambda}\left(\beta + \frac{\alpha(\eta - r)}{\sqrt{\delta^2 + (\eta - r)^2}} \right) \\\\
\varphi &= \sqrt{\alpha^2 - \beta^2} \\\\
\alpha &= \frac{3\sqrt{a}}{\sigma^2 b} \\\\
\beta &= \frac{3S}{\sigma b} \\\\
\eta &= \mu - \frac{3S\sigma}{a} \\\\
\delta &= 3\sigma\frac{\sqrt{b}}{a} \\\\
a &= 3K - 4S^2 - 9 \\\\
b &= 3K - 5S^2 - 9 \\
\end{align}

In [ ]:
# Compute the generalized skew-kurtosis Sharpe Ratio as explained in 
# "Portfolio performance evaluation with generalized Sharpe ratios: 
# Beyond the mean and variance" (Zakamouline and Koekebakker, 2008).

# returns_data: DataFrame with Timestamp indices and daily pct change.
# window: size of rolling window.
def asksr(returns_data, window=30):
    
    # Create a dataframe in which to store ASKSR and SR values and
    # model assumptions required to verify the validity of ASKSR values.
    sharpes = pd.DataFrame(
        columns=['ASKSR', 
                 'SR', 
                 'Delta > 0', 
                 'Alpha > 0', 
                 'Beta/alpha < 1',
                 'Kurtosis/Skewness'])
    
    # Use the U.S. Treasury long-term bill rate of return as the "risk free rate"
    risk_free = q.get('USTREASURY/REALLONGTERM', start_date=start, end_date=end).dropna().pct_change()
    risk_free_rolling = risk_free.rolling(window=30, min_periods=4).mean().dropna().values.tolist()
    
    # Generate a Rolling object over which to compute the rolling statistical moments.
    rolling = returns_data.rolling(window=30, min_periods=4)
    mean = rolling.mean().dropna()
    var = rolling.var().dropna()
    std = rolling.std().dropna()
    skew = rolling.apply(lambda x : stats.skew(x), raw=True).dropna()
    kurt = rolling.apply(lambda x : stats.kurtosis(x), raw=True).dropna()

    # For each set of rolling values per date, compute the parameters describing ASKSR.
    for m, v, st, sk, ku, r, date in zip(mean, var, std, skew, kurt, risk_free_rolling, returns_data.index):
        
        a = 3*ku - 4*(sk**2)-9
        b = 3*ku - 5*(sk**2)-9
        alpha = (3*c.sqrt(a))/(v*b)
        beta = (3*sk)/(c.sqrt(v)*b)
        eta = m - (3*sk*c.sqrt(v))/(a)
        sigma = 3*c.sqrt(v)*(c.sqrt(b)/a)
        phi = c.sqrt(alpha**2 - beta**2)
        delta = (m - eta)*(phi/beta)
        a_star = beta + (alpha*(eta - r[0]))/(c.sqrt(delta**2 + (eta - r[0])**2))

        # cmath library is used to handle complex values (unsure if this should be expected).
        ASKSR = c.sqrt(2*(a_star*(eta - r[0]) - delta*(
            phi - c.sqrt(alpha**2 - (beta - a_star)**2))))*c.sqrt(window)
        ASKSR_mag = ASKSR.real**2 + ASKSR.imag**2
        SR = ((m - r[0])/(np.sqrt(v) - r[0]))*np.sqrt(window)
        
        # Generate a Pandas Series to append to the sharpes dataframe.
        row = pd.Series([float('%.3f'%(ASKSR_mag)), 
                        float('%.3f'%(SR)), 
                        str(delta.real > 0),
                        str(alpha.real > 0), 
                        str((np.abs(beta)/alpha) < 1), 
                        str(ku > (5/3)*(sk**2) + 3)], 
                       index=sharpes.columns).rename(date)
        sharpes = sharpes.append(row)
    
    return sharpes

In [ ]:
# Create a dataframe in which to store pct returns of all stocks in port.
p = pd.DataFrame()

# For each stock, extract the percent change and add it to p.
for tick, data in port_rets.items():
    p[tick] = data['pct change']

# Calculate the net return of the portfolio after each trading day.
p['net'] = p.sum(axis=1)

In [ ]:
p.head()

In [ ]:
s = asksr(p['net'])

In [ ]:
s.describe()

Notice how the ASKSR readings tend to be much greater than the standard SR and has a tendency to become extremely large.

In [ ]:
s.plot(figsize=(15,5));

# Technical Indicators

## ZUO (1yr and 1mo)

In [ ]:
stock = 'ZUO'

# Enter start date
start = '2018-04-21'

# Default set to today
end = dt.datetime.today().strftime('%Y-%m-%d')
ti = ti_dct[0]

macd, macd_meta = ti.get_macd(stock)
macd = pd.DataFrame(macd)
macd.index = pd.to_datetime(macd.index)

rsi, rsi_meta = ti.get_rsi(stock)
rsi = pd.DataFrame(rsi)
rsi.index = pd.to_datetime(rsi.index)

boll, boll_meta = ti.get_bbands(stock, time_period=60)
boll = pd.DataFrame(boll)
boll.index = pd.to_datetime(boll.index)

ad, ad_meta = ti.get_ad(stock)
ad = pd.DataFrame(ad)
ad.index = pd.to_datetime(ad.index)

obv, obv_meta = ti.get_obv(stock)
obv = pd.DataFrame(obv)
obv.index = pd.to_datetime(obv.index)

In [ ]:
size = (15,3)
port['ZUO']['close'][start:end].plot(title='Close Price', figsize=size)
macd[start:end].plot(title= 'MACD', figsize=size);
rsi[start:end].plot(title='RSI', figsize=size);
boll[start:end].plot(title='Bollinger Bands', figsize=size);
ad[start:end].plot(title='Chaikin A/D Line', figsize=size);
obv[start:end].plot(title='On-Balance Volume', figsize=size);

In [ ]:
start = '2019-03-21'

size = (15,3)
port['ZUO']['close'][start:end].plot(title='Close Price', figsize=size)
macd[start:end].plot(title= 'MACD', figsize=size);
rsi[start:end].plot(title='RSI', figsize=size);
boll[start:end].plot(title='Bollinger Bands', figsize=size);
ad[start:end].plot(title='Chaikin A/D Line', figsize=size);
obv[start:end].plot(title='On-Balance Volume', figsize=size);

# Under Construction

## Efficiency Ratio

In [ ]:
np.var(mean-r)

In [ ]:
def eff_rat(symbol, period=7):
    close_data = port[symbol]['close']
    close
    
    return pd.DataFrame()

In [ ]:
close_data = port['EPD']['close']

abs_change = close_data.diff().resample('7D').sum()
#summed_data = close_data.resample('7D')